In [40]:
import xlwings as  xw
import pandas as pd
import datetime

In [47]:
def get_reshaped_record(record):
    
    dates = set(record['Date'])
    descs = record['Desc']
    record = record.set_index(['Date','Desc'])
    data = []
    for d in dates:
        row = [d]
        for desc in descs:
            if (d, desc) in record.index:
                row.append(record.loc[d, desc]['Fixing'])
            else:
                row.append(-1)
        data.append(row)
    cols = ['DATE'] + list(descs)
    new_df = pd.DataFrame(data, columns = cols )
    new_df = new_df.set_index('DATE')
    return new_df

def update_history(history, new_record):
    for d in new_record.index:
        for rate in new_record.columns:
            if new_record.at[d, rate] != -1:
                history.at[d, rate] = new_record.at[d, rate]
    history.fillna(-1, inplace=True)

In [48]:
%run ./EuriborEurirsScraper.ipynb
record = pd.concat([df_ESTER, df_EURIBOR, df_EURIRS]).reset_index(drop=True)

excel_name = 'fixings.xlsx'
sheet = "fixing_history"

In [49]:
new_record = get_reshaped_record(record)
history = pd.read_excel('fixings.xlsx')
history = history.set_index('DATE')
update_history(history, new_record)

In [51]:
with pd.ExcelWriter(excel_name, date_format = 'yyyy-mm-dd', datetime_format = 'yyyy-mm-dd') as writer:
    history.sort_index(ascending = False).to_excel(writer, sheet_name = sheet, index=True )

PermissionError: [Errno 13] Permission denied: 'fixings.xlsx'

In [50]:
history

,ESTER,EURIBOR 1W,EURIBOR 1M,EURIBOR 3M,EURIBOR 6M,EURIBOR 12M,IRS 1Y,IRS 2Y,IRS 3Y,IRS 4Y,...,IRS 9Y,IRS 10Y,IRS 11Y,IRS 12Y,IRS 15Y,IRS 20Y,IRS 25Y,IRS 30Y,IRS 40Y,IRS 50Y
DATE,,,,,,,,,,,,,,,,,,,,,
2022-05-04,-0.00585,-1.00000,-1.00000,-1.00000,-1.00000,-1.00000,0.0030,0.0100,0.0128,0.0142,...,0.0173,0.0178,0.0182,0.0185,0.0189,0.0178,0.0162,0.0147,0.0126,0.0111
2022-05-03,-0.00579,-0.00566,-0.00543,-0.00425,-0.00198,0.00225,0.0028,0.0099,0.0125,0.0138,...,0.0170,0.0176,0.0180,0.0183,0.0187,0.0178,0.0163,0.0148,0.0129,0.0115
2022-05-02,-0.00583,-0.00568,-0.00527,-0.00416,-0.00204,0.00213,0.0027,0.0097,0.0123,0.0138,...,0.0171,0.0176,0.0181,0.0185,0.0189,0.0183,0.0167,0.0153,0.0135,0.0122
2022-04-29,-0.00586,-0.00566,-0.00538,-0.00429,-0.00226,0.00166,0.0027,0.0098,0.0124,0.0138,...,0.0169,0.0174,0.0178,0.0182,0.0187,0.0179,0.0164,0.0150,0.0134,0.0120
2022-04-28,-0.00583,-0.00571,-0.00522,-0.00438,-0.00242,0.00118,0.0022,0.0090,0.0118,0.0132,...,0.0164,0.0169,0.0175,0.0179,0.0184,0.0177,0.0162,0.0148,0.0132,0.0119
2022-04-27,-0.00583,-0.00569,-0.00520,-0.00445,-0.00251,0.00109,0.0016,0.0080,0.0108,0.0120,...,0.0153,0.0160,0.0164,0.0168,0.0174,0.0168,0.0154,0.0140,0.0122,0.0109
2022-04-26,-0.00582,-0.00576,-0.00513,-0.00430,-0.00246,0.00111,0.0017,0.0080,0.0109,0.0120,...,0.0151,0.0156,0.0160,0.0165,0.0170,0.0163,0.0149,0.0135,0.0117,0.0104
